In [1]:
import numpy as np
from myopic import exp, MAP_SIZE
from multiprocessing import Pool
from tqdm.contrib.concurrent import process_map
from tqdm import tqdm
import tensorflow as tf

/Users/saumik/mambaforge/envs/gymtest/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-15 00:01:43.074848: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
seeds = 100000

In [3]:
res = process_map(exp, range(seeds), chunksize=1)
grids, actions = zip(*res)
grids, actions = np.array(grids), np.array(actions)

100%|██████████| 100000/100000 [03:22<00:00, 493.08it/s]


In [4]:
split = int(seeds*0.8)
x_train, x_test = grids[:split], grids[split:]
y_train, y_test = actions[:split], actions[split:]

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(3, MAP_SIZE, MAP_SIZE)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(4)
])
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])


In [6]:
model.fit(x_train, y_train, epochs=1)
model.evaluate(x_test, y_test, verbose=2)

2500/2500 [==============================] - 4s 1ms/step - loss: 0.5106 - accuracy: 0.7922
625/625 - 1s - loss: 0.4248 - accuracy: 0.8233 - 590ms/epoch - 944us/step


[0.4248158633708954, 0.8233000040054321]

In [7]:
y_pred = model.predict(x_test)

625/625 [==============================] - 1s 1ms/step


In [8]:
tf.nn.softmax(y_pred[-7]).numpy().round(2)

array([0.03, 0.  , 0.  , 0.97], dtype=float32)

In [9]:
grids[-7].astype(int)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 1, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [1, 0, 0]],

       [[1, 0, 0],
        [1, 0, 1],
        [0, 0, 1]]])

In [10]:
actions[-7]

3